In [1]:
%reset -f
import ROOT
ROOT.gErrorIgnoreLevel = ROOT.kWarning

ROOT.gStyle.SetLabelFont(42,'xy')
ROOT.gStyle.SetLabelSize(0.05,'xy')
ROOT.gStyle.SetTitleSize(0.07,'xy')
ROOT.gStyle.SetTitleOffset(0.75,'xy')

ROOT.gStyle.SetOptFit(1)


mggmu, mggsig = 0.1325, 0.0125 
gglims = [mggmu+isig*mggsig for isig in [-5,-3,3,5]]
pbs = [(1, 0.86), (2,0.89)]

%run -i eppi0_columns.py
%run -i eppi0_binning_scheme_v3.py
%run -i eppi0_bsa_side.py
%run -i eppi0_ALU_p1.py

def process(fname):
    df = ROOT.RDataFrame("h22",fname)
    df = define_eppi0_columns_using_proton(df)
    shards, df = define_eppi0_bins(df, fname)
    return shards

shards0 = process("../data/eppi0.inb.qa.root")
shards1 = process("../data/eppi0.outb.qa.root")
#shards1 = process("data/mc.root")

grbsa0 = get_bsas(shards0)
grbsa1 = get_bsas(shards1)


grbsa0side = get_bsas(shards0)
grbsa1side = get_bsas(shards1)
grsig0p1 = get_sigs(grbsa0side)
grsig1p1 = get_sigs(grbsa1side)


#######################
%reset_selective -f get_bsas
%run -i eppi0_bsa_ggfit.py
grbsa0ggfit = get_bsas(shards0)
grbsa1ggfit = get_bsas(shards1)
grsig0ggfit = get_sigs(grbsa0ggfit, 0.02)
grsig1ggfit = get_sigs(grbsa1ggfit, 0.02)

#######################
%reset_selective -f get_bsas
%run -i eppi0_bsa_raw.py
grbsa0raw = get_bsas(shards0)
grbsa1raw = get_bsas(shards1)
grsig0raw = get_sigs(grbsa0raw, -0.03)
grsig1raw = get_sigs(grbsa1raw, -0.03)




#######################
%reset_selective -f get_sigs
%run -i eppi0_ALU_p2.py
grsig0p2 = get_sigs(grbsa0side, 0.02)
grsig1p2 = get_sigs(grbsa1side, 0.02)

#######################
%reset_selective -f get_sigs
%run -i eppi0_ALU_p2_fixed.py
grsig0p2f = get_sigs(grbsa0side, 0.06)
grsig1p2f = get_sigs(grbsa1side, 0.06)

#######################
%reset_selective -f get_sigs
%run -i eppi0_ALU_p3_fixed.py
grsig0p3f = get_sigs(grbsa0side, 0.04)
grsig1p3f = get_sigs(grbsa1side, 0.04)

#get_sigs(grbsa0)
#get_sigs(grbsa1)



grsig,grbsa = {},{}
for kk in grsig0p1:
    grsig[kk] = [("raw", grsig0raw[kk], grsig1raw[kk]), ("sideband", grsig0p1[kk],grsig1p1[kk]), ("fitting", grsig0ggfit[kk],grsig1ggfit[kk])]
    #grsig[kk] = [("1 par", grsig0p1[kk],grsig1p1[kk]), ("2 pars", grsig0p2[kk],grsig1p2[kk]), ("fixed #sigma_{TT}", grsig0p3f[kk],grsig1p3f[kk])]
    grsig[kk] = [("INb", grsig0p3f[kk]), ("OUTb", grsig1p3f[kk])]
for kk in grbsa0side:
    grbsa[kk] = [("INb", grbsa0side[kk]), ("OUTb", grbsa1side[kk])]
    grbsa[kk] = [("raw", grbsa0raw[kk]), ("sideband", grbsa0side[kk]), ("fitting", grbsa0ggfit[kk])]
    grbsa[kk] = [("sideband", grbsa0side[kk])]
    #grbsa[kk] = [("sideband", grbsa0[kk])]

Welcome to JupyROOT 6.24/06


In [3]:
from collections import defaultdict
grGK = defaultdict(lambda: ROOT.TGraph())

ikin = 0
with open("../vpk/gk_v3.dat") as ff:
    for line in ff:
        #q2,xb,tt,eps,alu0,alu = [float(vv) for vv in line.split()]
        q2,xb,tt,alu,eps = [float(vv) for vv in line.split()]
        gr = grGK[(q2,xb)]
        if gr.GetN()==0:
            grGK[(ikin,)] = gr
            print(ikin,q2,xb)
            ikin+=1
        gr.SetPoint(gr.GetN(), -tt, alu/math.sqrt(eps*(1-eps)))

0 2.6 0.29
1 3.03 0.39
2 4.68 0.37
3 4.15 0.51
4 6.7 0.57


In [4]:
lat = ROOT.TLatex()
lat.SetTextSize(0.05)
c1 = ROOT.TCanvas("c1","c1",1100,800)
c1.SetGrid()
c1.SetMargin(0.12,0.001,0.12,0.001)
c1.Print('grsig.pdf[')
for kk in sorted(grsig):
    fr = c1.DrawFrame(0,-0.04,2,0.312)
    
    #q2,xb = shards0[kk]
    #q2,xb = q2.GetValue(), xb.GetValue()
    #print(kk, q2,xb)
    #fr.SetTitle("Q^{{2}}={:.2f}, x_{{B}}={:.3f}".format(q2,xb))
    fr.SetTitleSize(0.06,"xy")
    fr.SetTitleOffset(0.8,"x")
    fr.SetTitleOffset(0.9,"y")
    fr.SetTitle(";-t [GeV^{2}];d#sigma_{LT'}/d#sigma_{0}")
    fr1 = fr.Clone("fr")
    fr.SetAxisColor(14,"xy")
    fr1.Draw("sameaxis")

    grGK[kk].Draw("C")
    grGK[kk].SetLineColor(4)
    grGK[kk].SetLineWidth(3)
    cols = [0,1,2,4,6,7]
    igr = 0;
    for grs in grsig[kk]:
        grs = list(grs)
        name = grs.pop(0)
        igr += 1
        for gr in grs:
            gr.Draw("P")
            gr.SetMarkerStyle(19+igr)
            gr.SetMarkerColor(cols[igr])
            gr.SetLineColor(cols[igr])
            gr.SetMarkerSize(2)
            gr.SetLineWidth(2)
            q2,xb = gr.qmean,gr.xmean

        lat.DrawLatexNDC(0.45,1-igr*0.08,"{}: Q^{{2}}={:.2f} [GeV^{{2}}], x_{{B}}={:.2f}".format(name,q2,xb)).SetTextColor(gr.GetLineColor())
        #if igr==1:
        #    lat.DrawLatexNDC(0.45,1-igr*0.08,"Q^{{2}}={:.2f} [GeV^{{2}}], x_{{B}}={:.2f}".format(q2,xb)).SetTextColor(gr.GetLineColor())
    c1.Print('grsig.pdf')
c1.Print('grsig.pdf]')
#!mogrify -format png -trim -transparent white grsig.pdf

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [6]:
c2 = ROOT.TCanvas("c2","c2",2400,350)
#c2.Divide(3,1,0.0001,0.0001)
c2.Divide(3,1,0,0)
c2.Print("grbsa.pdf[")
frs=[]
for tpl in sorted(grbsa0side):
    iqx,itt = tpl
    fr = c2.cd(itt+1).DrawFrame(0,-0.28,360,0.28)
    c2.cd(itt+1).SetGrid()
    fr1 = fr.Clone(fr.GetName()+".itt"+str(itt))
    fr1.Draw("same axis")
    frs.append(fr1)
    fr.SetAxisColor(16,'xy')
    fr.SetTitle(";#phi;BSA")
    fr.SetLabelSize(0.08,"xy")
    fr.SetTitleSize(0.08,"xy")
    fr.SetTitleOffset(0.5,"x")
    fr.SetTitleOffset(0.6,"y")
    #fr.SetTitle("iqx={}, itt={}".format(iqx,itt))
    
    igr = 0
    for name,gr in grbsa[tpl]:
        igr+=1
        gr.UseCurrentStyle()
        gr.SetMarkerStyle(20)
        gr.SetMarkerSize(2)
        gr.SetMarkerColor(igr)
        gr.SetLineColor(igr)
        for f1 in gr.GetListOfFunctions():
            f1.SetLineColor(gr.GetLineColor())
            f1.SetLineWidth(1)
        gr.Draw("P")

    if itt==2:
        c2.Print("grbsa.pdf")

c2.Print("grbsa.pdf]")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2
